In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"}
])
print(response)

content='こんにちは、ジョンさん！お会いできて嬉しいです。何かお手伝いできることはありますか？' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b5f68-9cfc-7911-a66e-79f0dd6b0786-0' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    streaming=True
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"},

])
print(response)


content='こんにちは、ジョンさん！お会いできて嬉しいです。何かお手伝いできることはありますか？\n' additional_kwargs={} response_metadata={'safety_ratings': [], 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'model_provider': 'google_genai'} id='lc_run--019b5ef9-1c18-7d41-932a-5407cb6d0492' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    response_mime_type="application/json"
)

response = client.invoke([
    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
])

print(response.content)

```json
[
  {"name": "佐藤太郎", "age": "50", "city": "東京都"},
  {"name": "鈴木花子", "age": "30", "city": "大阪府"}
]
```


In [ ]:
# ====================================
# Function Calling 入門
# ====================================
# AIが「どの関数を呼ぶべきか」を判断し、引数を生成する機能

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

# 1. ツール（関数）を定義
@tool
def get_weather(city: str) -> str:
    """指定した都市の天気を取得する"""
    # 実際はAPIを呼ぶが、ここではダミーデータ
    weather_data = {
        "東京": "晴れ 25℃",
        "大阪": "曇り 22℃",
        "名古屋": "雨 18℃"
    }
    return weather_data.get(city, f"{city}の天気情報はありません")

@tool
def calculate(expression: str) -> str:
    """数式を計算する"""
    try:
        result = eval(expression)
        return f"計算結果: {result}"
    except:
        return "計算できませんでした"

# 2. LLMにツールをバインド
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_tools = llm.bind_tools([get_weather, calculate])

# 3. 質問してみる
response = llm_with_tools.invoke("大分県の天気は？")
#今まではresponse = llm.invoke{[
#    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
#    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
#]}


print("=== AIの判断 ===")
print(f"呼び出すツール: {response.tool_calls}")
print()

# 4. ツールを実際に実行
if response.tool_calls:
    tool_call = response.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"=== ツール実行 ===")
    print(f"ツール名: {tool_name}")
    print(f"引数: {tool_args}")

    # ツールを実行
    if tool_name == "get_weather":
        result = get_weather.invoke(tool_args)
    elif tool_name == "calculate":
        result = calculate.invoke(tool_args)

    print(f"結果: {result}")

=== AIの判断 ===
呼び出すツール: [{'name': 'get_weather', 'args': {'city': '大分県'}, 'id': 'deff81d0-02a0-47a1-a003-9a24db87087c', 'type': 'tool_call'}]

=== ツール実行 ===
ツール名: get_weather
引数: {'city': '大分県'}
結果: 大分県の天気情報はありません


In [ ]:
import json

def get_current_weather(location,unint="fahrenheit"):
    if "tokyo" in location.lower():
        return {
            "location": "Tokyo",
            "temperature": "22",
            "unit": unint,
            "forecast": "sunny"
        }
    elif "osaka" in location.lower():
        return {
            "location": "Osaka",
            "temperature": "25",
            "unit": unint,
            "forecast": "cloudy"
        }
    else:
        return {
            "location": location,
            "temperature": "unknown",
            "unit": unint,
            "forecast": "unknown"
        }

In [ ]:
# ====================================
# Function Calling × SQL
# ====================================
# 自然言語からSQLを生成・実行する

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import sqlite3

# 1. インメモリDBを作成してサンプルデータを投入
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        city TEXT,
        department TEXT
    )
""")

# サンプルデータ
sample_users = [
    ("山田太郎", 30, "東京", "営業部"),
    ("鈴木花子", 25, "大阪", "開発部"),
    ("佐藤次郎", 28, "名古屋", "営業部"),
    ("田中美咲", 35, "東京", "人事部"),
    ("高橋健一", 42, "福岡", "開発部"),
    ("伊藤さくら", 23, "東京", "開発部"),
]

cursor.executemany("INSERT INTO users (name, age, city, department) VALUES (?, ?, ?, ?)", sample_users)
conn.commit()

print("=== データベース作成完了 ===")
print("テーブル: users (id, name, age, city, department)")
print(f"データ件数: {len(sample_users)}件")
print()

# 全データ表示
cursor.execute("SELECT * FROM users")
for row in cursor.fetchall():
    print(row)

In [ ]:
# 2. SQLを実行するツールを定義
@tool
def execute_sql(query: str) -> str:
    """
    SQLクエリを実行してデータベースから情報を取得する。

    テーブル: users
    カラム: id, name(名前), age(年齢), city(都市), department(部署)

    例:
    - SELECT * FROM users WHERE city = '東京'
    - SELECT * FROM users WHERE age >= 30
    - SELECT department, COUNT(*) FROM users GROUP BY department
    """
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        if results:
            columns = [desc[0] for desc in cursor.description]
            output = f"カラム: {columns}\n"
            for row in results:
                output += f"{row}\n"
            return output
        return "該当するデータがありませんでした"
    except Exception as e:
        return f"エラー: {e}"

# 3. LLMにツールをバインド
llm_sql = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_sql = llm_sql.bind_tools([execute_sql])

print("=== SQLツール設定完了 ===")

In [ ]:
# 4. 自然言語で質問してみる
def ask_database(question: str):
    """自然言語でDBに質問する"""
    print(f"質問: {question}")
    print("-" * 40)

    response = llm_with_sql.invoke(question)

    if response.tool_calls:
        tool_call = response.tool_calls[0]
        sql_query = tool_call["args"]["query"]

        print(f"生成されたSQL: {sql_query}")
        print("-" * 40)

        result = execute_sql.invoke({"query": sql_query})
        print(f"結果:\n{result}")
    else:
        print(f"AIの回答: {response.content}")

    print("=" * 50)
    print()

# いくつかの質問を試す
ask_database("東京に住んでいる人を全員教えて")

In [ ]:
# 他の質問も試してみよう
ask_database("30歳以上の社員は誰？")
ask_database("部署ごとの人数を教えて")
ask_database("開発部で一番若い人は？")

In [18]:
prompt ='''\
以下の料理のレシピを教えてください。

料理名: """
{dish}
"""
'''

def get_recipe(dish_name: str) -> str:
    response = llm.invoke([
        {"role": "user", "content": prompt.format(dish=dish_name)}
    ])
    return response.content

get_recipe("カレーライス")
#prompt.format(dish="田中")

'はい、カレーライスのレシピですね。家庭料理の定番ですが、色々な作り方があります。ここでは、基本的な作り方と、ちょっと美味しくなるコツをいくつかご紹介します。\n\n**材料 (4人分)**\n\n*   牛肉または豚肉 (カレー用) : 300g\n*   玉ねぎ : 大2個\n*   じゃがいも : 中3個\n*   にんじん : 中1本\n*   カレールー : 1箱 (8〜12皿分)\n*   サラダ油 : 大さじ2\n*   水 : カレールーの箱の指示通り\n*   ご飯 : 4人分\n\n**美味しくなるコツ**\n\n*   **肉:** 豚バラ肉を使うとコクが出ます。牛肉を使うなら、煮込み用のすね肉などがおすすめです。\n*   **玉ねぎ:** じっくり炒めることで甘みとコクがアップします。飴色になるまで炒めると、より美味しくなります。\n*   **じゃがいも:** メークインを使うと煮崩れしにくいです。\n*   **隠し味:**\n    *   チョコレートひとかけら（ビターチョコレートがおすすめ）\n    *   インスタントコーヒー小さじ1/2\n    *   ソース大さじ1\n    *   ケチャップ大さじ1\n    *   ヨーグルト大さじ2\n    *   赤ワイン大さじ2\n    *   醤油小さじ1\n\n**作り方**\n\n1.  **下準備:**\n    *   肉は大きければ一口大に切る。\n    *   玉ねぎは薄切りにする。\n    *   じゃがいも、にんじんは皮をむき、一口大に切る。じゃがいもは水にさらしてアクを抜く。\n\n2.  **炒める:**\n    *   鍋にサラダ油を熱し、肉を炒める。色が変わったら、玉ねぎを加えて炒める。\n    *   玉ねぎがしんなりしたら、じゃがいも、にんじんを加えて炒める。全体に油が回ったら、水を加える。\n\n3.  **煮込む:**\n    *   沸騰したらアクを取り、弱火にして、じゃがいもとにんじんが柔らかくなるまで煮込む(約15〜20分)。\n\n4.  **ルーを溶かす:**\n    *   一旦火を止め、カレールーを割り入れ、溶けるまでよく混ぜる。\n    *   ルーが完全に溶けたら、再び弱火でとろみがつくまで煮込む。焦げ付か

In [24]:
# ====================================
# LangSmith トレース例（langchain-google-vertexai使用）
# ====================================

from dotenv import load_dotenv
load_dotenv()  # LangSmithの設定を読み込む

from langchain_google_genai import ChatGoogleGenerativeAI

# Vertex AI経由でGeminiを使用
llm_vertex = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 基本的な使い方
response = llm_vertex.invoke("こんにちは！今日の気分はどう？")
print("=== 基本的な呼び出し ===")
print(response.content)
print()

# メッセージ形式
response2 = llm_vertex.invoke([
    {"role": "system", "content": "あなたは関西弁で話すアシスタントです。"},
    {"role": "user", "content": "自己紹介してください"}
])
print("=== システムプロンプト付き ===")
print(response2.content)

=== 基本的な呼び出し ===
こんにちは！今日の気分はとても良いですよ！ 😊 天気も良いですし、あなたとお話できるのが嬉しいです。あなたはどんな気分ですか？

=== システムプロンプト付き ===
へい、毎度！わいはあんたの頼れるアシスタントや。名前はまだないんやけど、せやけど、なんでも聞いてや。知ってることなら、おもろい関西弁で教えたるさかい！よろしく頼むで！
